In [1]:
#Note complete, building XGBoostModel so far just been switching data to correct format.

In [2]:
from numpy import loadtxt

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
from xgboost import XGBClassifier

In [6]:
train = pd.read_csv(r"C:\Users\comic\NFL-Predictor\Data\Train-Data\Compiled\testNOPERCENTAGES.csv")#Data we will train with preview
train.head()

,T1,T1 Outcome,T1 Score,T1 Location,T1 Avg Points,T1 Avg Yards,T1 NPY/A,T1 NRY/A,T1 3rd,T1 4th,...,T2 NPY/A,T2 NRY/A,T2 3rd,T2 4th,T2 Avg Points Def,T2 Avg Yards Def,T2 NPY/A Def,T2 NRY/A Def,T2 3rd Def,T2 4th Def
0,San Francisco 49ers,1,16,Away,22.9375,356.3125,5.8,4.6,0.52,0.62,...,6.9,3.9,0.39,0.36,17.4375,309.3125,5.4,4.5,0.37,0.38
1,Atlanta Falcons,0,34,Away,25.1250,345.9375,6.1,4.5,0.42,0.60,...,6.0,4.3,0.39,0.54,20.5000,311.5625,5.2,4.8,0.34,0.38
2,Buffalo Bills,0,31,Home,23.6875,349.4375,6.0,4.1,0.43,0.58,...,6.6,4.0,0.40,0.33,21.0000,341.4375,6.0,4.6,0.44,0.56
3,San Diego Chargers,1,34,Away,20.8125,332.8125,5.7,4.6,0.37,0.33,...,5.5,4.1,0.39,0.37,28.5000,329.0625,6.8,4.0,0.42,0.50
4,Miami Dolphins,1,49,Home,23.6250,337.0000,6.0,4.7,0.38,0.30,...,5.0,4.1,0.30,0.40,28.1875,382.3125,6.7,4.0,0.45,0.73


In [7]:
#Added
#train = train
#Added


train = train.drop(["T1", "T1 Score", "T2", "T2 Score", "T2 Outcome"], axis =1)
#Remove columns don't want considered.

#Turn Home into a 1 value and Away into a 0 value to get data into good format for LightGBM
train=train.replace(to_replace="Home",value="1")
train=train.replace(to_replace="Away",value="0")
print(train)


      T1 Outcome T1 Location  T1 Avg Points  T1 Avg Yards  T1 NPY/A  T1 NRY/A  \
0              1           0        22.9375      356.3125       5.8       4.6   
1              0           0        25.1250      345.9375       6.1       4.5   
2              0           1        23.6875      349.4375       6.0       4.1   
3              1           0        20.8125      332.8125       5.7       4.6   
4              1           1        23.6250      337.0000       6.0       4.7   
...          ...         ...            ...           ...       ...       ...   
4347           1           1        24.3125      374.9375       6.1       4.5   
4348           0           0        18.1250      336.1875       6.2       4.2   
4349           0           1        17.5625      299.6875       5.5       4.3   
4350           1           0        26.7500      372.6250       7.5       4.7   
4351           0           1        19.3750      312.6250       6.1       4.4   

      T1 3rd  T1 4th  T1 Av

In [8]:
train = train[['T1 Location', 'T1 Avg Points', 'T1 NPY/A', 'T1 NRY/A','T1 3rd', 'T1 4th', 'T1 Avg Points Def','T1 Avg Yards Def','T1 NPY/A Def','T1 NRY/A Def','T1 3rd Def','T1 4th Def','T2 Location','T2 Avg Points','T2 Avg Yards','T2 NPY/A','T2 NRY/A','T2 3rd','T2 4th','T2 Avg Points Def','T2 Avg Yards Def','T2 NPY/A Def','T2 NRY/A Def','T2 3rd Def','T2 4th Def', 'T1 Outcome']]
print(train)

     T1 Location  T1 Avg Points  T1 NPY/A  T1 NRY/A  T1 3rd  T1 4th  \
0              0        22.9375       5.8       4.6    0.52    0.62   
1              0        25.1250       6.1       4.5    0.42    0.60   
2              1        23.6875       6.0       4.1    0.43    0.58   
3              0        20.8125       5.7       4.6    0.37    0.33   
4              1        23.6250       6.0       4.7    0.38    0.30   
...          ...            ...       ...       ...     ...     ...   
4347           1        24.3125       6.1       4.5    0.45    0.59   
4348           0        18.1250       6.2       4.2    0.37    0.44   
4349           1        17.5625       5.5       4.3    0.36    0.56   
4350           0        26.7500       7.5       4.7    0.40    0.88   
4351           1        19.3750       6.1       4.4    0.40    0.58   

      T1 Avg Points Def  T1 Avg Yards Def  T1 NPY/A Def  T1 NRY/A Def  ...  \
0               21.9375          322.3750           6.0           4.2

In [9]:
#train.to_csv('xgBoostTrainDataOutcomeLast.csv',index=False) #Use this if needed to get data in correct format. 

In [10]:
dataset = pd.read_csv(r"C:\Users\comic\NFL-Predictor\Data\Train-Data\XGBoostData\xgBoostTrainData.csv", header=None,skiprows=[0])
dataset.head()

#dataset.to_csv('xgBoostTrainDataNoHeadersOutcomeLast.csv',index=False)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1,0,22.9375,356.3125,5.8,4.6,0.52,0.62,21.9375,322.3750,...,6.9,3.9,0.39,0.36,17.4375,309.3125,5.4,4.5,0.37,0.38
1,0,0,25.1250,345.9375,6.1,4.5,0.42,0.60,19.6250,333.3750,...,6.0,4.3,0.39,0.54,20.5000,311.5625,5.2,4.8,0.34,0.38
2,0,1,23.6875,349.4375,6.0,4.1,0.43,0.58,24.8125,324.3125,...,6.6,4.0,0.40,0.33,21.0000,341.4375,6.0,4.6,0.44,0.56
3,1,0,20.8125,332.8125,5.7,4.6,0.37,0.33,22.9375,377.1250,...,5.5,4.1,0.39,0.37,28.5000,329.0625,6.8,4.0,0.42,0.50
4,1,1,23.6250,337.0000,6.0,4.7,0.38,0.30,18.8125,291.0000,...,5.0,4.1,0.30,0.40,28.1875,382.3125,6.7,4.0,0.45,0.73


In [11]:
#dataset1 = loadtxt(r"C:\Users\comic\NFL-Predictor\Data\Train-Data\XGBoostData\xgBoostTrainDataNoHeadersOutcomeLast.csv", delimiter=",")
dataset1 = loadtxt(r"C:\Users\comic\NFL-Predictor\Data\Train-Data\XGBoostData\xgBoostTrainDataNoHeaders.csv", delimiter=",")

In [12]:
X = dataset1[:,0:8]
Y = dataset1[:,8]

In [13]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [14]:
model = XGBClassifier()
model.fit(X_train, y_train)

c:\users\comic\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:01:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [16]:
# make predictions for test data
y_pred = model.predict(X_test)
#predictions = [round(value) for value in y_pred]


In [17]:
# evaluate predictions
#accuracy = accuracy_score(y_test, y_pred, normalize=False)
#accuracy_score(y_test, y_pred.round(), normalize=False)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

#x_train= x_train.reshape(-1, 1)
#y_pred= y_pred.reshape(-1, 1)
#y_test = y_test.reshape(-1, 1)


#score = model.score(X_test, y_pred.round())

pred = model.predict(X_test)
print(pred)



#predictions = gbm.predict(test_X)

[19.     24.1875 15.3125 ... 20.5625 16.25   19.6875]
